In [2]:
import re
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import math
import random
import os
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM

Using TensorFlow backend.


In [3]:
#using pretrained glove embeddings to embed words
embeddings_index = {}
embedding_dim = 100 #change to other dim as well
f = open(os.path.join('./glove.6B', 'glove.6B.100d.txt')) #TODO try 300 dimensions
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [4]:
#Preprocessing xml files to x train and y train data
import xml.etree.ElementTree as ET

path = './en/'
all_texts = []
y_train = []
x_train = []
max_text_length = 500
for filename in os.listdir(path):    
    root = ET.parse(path + filename).getroot()
    #TODO add for other classifications, ie. age_group and multi-class 
    y = 0
    if(root.attrib['gender'] == 'male'):
        y = 1
       
    for text in root.findall('conversations/conversation'):
        removed_tags = re.sub(r'<[^>]*>', '', str(text.text), flags=re.MULTILINE).lower()
        #TODO remove http 
        #removed_url = re.sub(r';http:[^\S]*\.php', '', removed_tags, flags=re.MULTILINE)
        #x_train.append(np.array(removed_tags.split()))        
        all_texts.append(removed_tags.split())            
        y_train.append(y)  

In [6]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(all_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index)) #3940 with the small train set

#can use x_train and y_train
for text in all_texts:
    text_ids = []
    for word in text:            
        word_id = word_index.get(word, -1)
        if word_id != -1:
            text_ids.append(word_id)
    x_train.append(text_ids)        
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
x = np.array(x_train)

#TODO pad sequence length to max_text_length
x_train = sequence.pad_sequences(x_train, maxlen=max_text_length, padding='post')      

Found 1543978 unique tokens.


In [7]:
#building embedding matrix
def get_embedding_layer(embedding_dim):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

    #embedding layer
    embedding_layer = Embedding(len(word_index) + 1, embedding_dim, weights=[embedding_matrix],
                                input_length=max_text_length, trainable=False)
    return embedding_layer

In [8]:
def build_model(embedding_dim, lstm_units, dropout):
    embedding_layer = get_embedding_layer(embedding_dim)
    model = Sequential()
    model.add(embedding_layer)
    model.add(LSTM(lstm_units))
    model.add(Dropout(dropout))
    model.add(Dense(2, activation='softmax'))
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])#try rmsprop
    return model    

In [8]:
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])#try rmsprop
print(model.summary())

model.fit(x_train, y_train, epochs=3, batch_size=512)#TODO try more epochs at least 100
model.save('./models/bs_512_em_100_model_3e.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          154397900 
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 154,478,502
Trainable params: 80,602
Non-trainable params: 154,397,900
_________________________________________________________________
None
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
16337

In [16]:
model2 = Sequential()
model2.add(embedding_layer)
model2.add(LSTM(100))
model2.add(BatchNormalization())#added
model2.add(Dropout(0.2))
model2.add(Dense(2, activation='softmax'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])#try rmsprop
print(model2.summary())

model2.fit(x_train[:10000], y_train[:10000], epochs=50, batch_size=512)#TODO try more epochs at least 100
model2.save('./models/small_bs_512_em_100_model_50e.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          154397900 
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 154,478,902
Trainable params: 80,802
Non-trainable params: 154,398,100
_________________________________________________________________
None
Epoch 1/50
10000/10000 [==============================] - 55s 5ms/step - loss: 0.8784 - acc: 0.5077
Epoch 2/50
10000/1

In [9]:
model3 = build_model(embedding_dim, lstm_units=100, dropout=0.2)
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])#try rmsprop
print(model3.summary)
model3.fit(x_train[:10000], y_train[:10000], epochs=10, validation_split=0.2, batch_size=512)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x7fd29efeaf28>>
Instructions for updating:
Use tf.cast instead.
Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 48s 6ms/step - loss: 0.6946 - acc: 0.5141 - val_loss: 0.7030 - val_acc: 0.5090
Epoch 2/10
8000/8000 [==============================] - 39s 5ms/step - loss: 0.6915 - acc: 0.5366 - val_loss: 0.6968 - val_acc: 0.5020
Epoch 3/10
8000/8000 [==============================] - 39s 5ms/step - loss: 0.6883 - acc: 0.5400 - val_loss: 0.6980 - val_acc: 0.4985
Epoch 4/10
8000/8000 [==============================] - 41s 5ms/step - loss: 0.6871 - acc: 0.5460 - val_loss: 0.6993 - val_acc: 0.4970
Epoch 5/10
8000/8000 [==============================] - 39s 5ms/step - loss: 0.68

In [10]:
model4 = build_model(embedding_dim, lstm_units=100, dropout=0.2)
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])#try rmsprop
print(model4.summary)
model4.fit(x_train[:10000], y_train[:10000], epochs=150, validation_split=0.2, batch_size=512)

<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x7fd29f2cadd8>>
Train on 8000 samples, validate on 2000 samples
Epoch 1/150
8000/8000 [==============================] - 60s 7ms/step - loss: 0.6938 - acc: 0.5274 - val_loss: 0.6948 - val_acc: 0.5020
Epoch 2/150
8000/8000 [==============================] - 59s 7ms/step - loss: 0.6927 - acc: 0.5246 - val_loss: 0.6952 - val_acc: 0.5050
Epoch 3/150
8000/8000 [==============================] - 60s 7ms/step - loss: 0.6878 - acc: 0.5414 - val_loss: 0.7034 - val_acc: 0.5040
Epoch 4/150
8000/8000 [==============================] - 60s 7ms/step - loss: 0.6860 - acc: 0.5455 - val_loss: 0.7010 - val_acc: 0.4970
Epoch 5/150
8000/8000 [==============================] - 58s 7ms/step - loss: 0.6847 - acc: 0.5481 - val_loss: 0.7063 - val_acc: 0.5035
Epoch 6/150
8000/8000 [==============================] - 59s 7ms/step - loss: 0.6826 - acc: 0.5563 - val_loss: 0.7032 - val_acc: 0.5060
Epoch 7/150
8000/8000 [=================

Epoch 60/150
8000/8000 [==============================] - 41s 5ms/step - loss: 0.2700 - acc: 0.8380 - val_loss: 1.9832 - val_acc: 0.5115
Epoch 61/150
8000/8000 [==============================] - 41s 5ms/step - loss: 0.2760 - acc: 0.8356 - val_loss: 1.8704 - val_acc: 0.5200
Epoch 62/150
8000/8000 [==============================] - 41s 5ms/step - loss: 0.2844 - acc: 0.8310 - val_loss: 1.7725 - val_acc: 0.5155
Epoch 63/150
8000/8000 [==============================] - 41s 5ms/step - loss: 0.2827 - acc: 0.8309 - val_loss: 1.9122 - val_acc: 0.5190
Epoch 64/150
8000/8000 [==============================] - 41s 5ms/step - loss: 0.2732 - acc: 0.8382 - val_loss: 1.7865 - val_acc: 0.5155
Epoch 65/150
8000/8000 [==============================] - 41s 5ms/step - loss: 0.2721 - acc: 0.8379 - val_loss: 1.8507 - val_acc: 0.5085
Epoch 66/150
8000/8000 [==============================] - 41s 5ms/step - loss: 0.2632 - acc: 0.8430 - val_loss: 1.7915 - val_acc: 0.5200
Epoch 67/150
8000/8000 [=================

In [11]:
model4.evaluate(x_train[-1000:], y_train[-1000:], verbose=1)

1000/1000 [==============================] - 2s 2ms/step


[2.190560478210449, 0.525]

In [1]:
model5 = build_model(embedding_dim, lstm_units=256, dropout=0.2)
model5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])#try rmsprop
model5.fit(x_train[:5000], y_train[:5000], epochs=200, validation_split=0.1, batch_size=256)

NameError: name 'build_model' is not defined